In [12]:
# sparql query a list of wikidata films and respective english wikipedia pages.

import pandas
import pydash
import requests

def value_extract(row, col):

    ''' Extract dictionary values. '''

    return pydash.get(row[col], 'value')    
    
def sparql_query(query, service):

    ''' Send sparql request, and formulate results into a dataframe. '''

    r = requests.get(service, params = {'format': 'json', 'query': query})
    data = pydash.get(r.json(), 'results.bindings')
    data = pandas.DataFrame.from_dict(data)
    for x in data.columns:    
        data[x] = data.apply(value_extract, col=x, axis=1)
    return data

dataframe = sparql_query("""
    SELECT DISTINCT ?film ?link WHERE {
        ?film wdt:P31 wd:Q11424
        OPTIONAL {
            ?link schema:about ?film.
            ?link schema:inLanguage "en".
            ?link schema:isPartOf <https://en.wikipedia.org/>
            }}""", 'https://query.wikidata.org/sparql')

print(len(dataframe))
dataframe.head()

266901


,film,link
0,http://www.wikidata.org/entity/Q108946,https://en.wikipedia.org/wiki/A_Few_Good_Men
1,http://www.wikidata.org/entity/Q108586,https://en.wikipedia.org/wiki/The_Proposal_(20...
2,http://www.wikidata.org/entity/Q109110,https://en.wikipedia.org/wiki/Marked_for_Death
3,http://www.wikidata.org/entity/Q82860,https://en.wikipedia.org/wiki/The_Italian_Brig...
4,http://www.wikidata.org/entity/Q82937,https://en.wikipedia.org/wiki/Agent_077:_From_...
